# TF Variables and matrix operations

* Intro: https://www.tensorflow.org/guide/variable
* Definitions: https://www.tensorflow.org/api_docs/python/tf/Variable

In [1]:
import tensorflow as tf
import numpy as np

Set `trainable` to false if this variable shouldn't participate in automatic gradient calculations. Example: a loop counter.

In [15]:
a = tf.Variable([0.0, 1.0], name='Fantasy', trainable=False)
a

<tf.Variable 'Fantasy:0' shape=(2,) dtype=float32, numpy=array([0., 1.], dtype=float32)>

# LinAlg

In [3]:
# Does tf recognize that scalar and 0-vector are the same? Yes.

a = tf.Variable(1.0)
b = tf.Variable([2.0])

print('a=',a)
print('b=', b)
c = tf.Variable(a+b)
print('c=', c.numpy())

a= <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
b= <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([2.], dtype=float32)>
c= [3.]


In [4]:
# What if number types are different? We need to convert them.

a = tf.Variable(1)
b = tf.Variable([2.0])

print('a=',a)
print('b=', b)
print(tf.Variable(tf.cast(a, tf.float32)+b))

a= <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1>
b= <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([2.], dtype=float32)>
<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([3.], dtype=float32)>


In [5]:
# Does it broadcast dimensions? Yes!
a = tf.Variable(1.0)
b = tf.Variable([2.0, 1.0])
c = tf.Variable([[0.0, 1.0], [1.0, -1.0]])

print('a=',a)
print('b=', b)
print('c=', c)
print('a+b=', tf.Variable(a+b))
print('a+c=', tf.Variable(a+c))
print('b+c=', tf.Variable(b+c))

a= <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
b= <tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([2., 1.], dtype=float32)>
c= <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[ 0.,  1.],
       [ 1., -1.]], dtype=float32)>
a+b= <tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([3., 2.], dtype=float32)>
a+c= <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [2., 0.]], dtype=float32)>
b+c= <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[2., 2.],
       [3., 0.]], dtype=float32)>


In [6]:
# As in numpy, 1-vectors aren't quite matrices.
# Note tho that they seem to be more column-like (2,) than row-like.
print(tf.Variable(            ([1.0, 2.0])))
print(tf.Variable(tf.transpose([1.0, 2.0])))

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1., 2.], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1., 2.], dtype=float32)>


In [7]:
# We can force the columness. And then transposition immediately makes sense
a = tf.Variable(tf.transpose([[1.0, 2.0]]))
print(a)
print(tf.Variable(tf.transpose(a)))

<tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[1.],
       [2.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[1., 2.]], dtype=float32)>


In [8]:
a = tf.Variable([[1,2], [3,4]])
b = tf.Variable([[0,1], [-1, 0]])
print('a*b=', a * b, '- Elementwise multiplication with broadcasting.')
print('b*a=', b * a, '- ...is symmetric obviously')
print('a times b=',tf.matmul(a,b))
print('b times a=', tf.matmul(b,a))
print('a @ b=', a @ b, '- same as tf.matmul()')
print('aprime times b=', tf.matmul(a, b, transpose_a=True))

a*b= tf.Tensor(
[[ 0  2]
 [-3  0]], shape=(2, 2), dtype=int32) - Elementwise multiplication with broadcasting.
b*a= tf.Tensor(
[[ 0  2]
 [-3  0]], shape=(2, 2), dtype=int32) - ...is symmetric obviously
a times b= tf.Tensor(
[[-2  1]
 [-4  3]], shape=(2, 2), dtype=int32)
b times a= tf.Tensor(
[[ 3  4]
 [-1 -2]], shape=(2, 2), dtype=int32)
a @ b= tf.Tensor(
[[-2  1]
 [-4  3]], shape=(2, 2), dtype=int32) - same as tf.matmul()
aprime times b= tf.Tensor(
[[-3  1]
 [-4  2]], shape=(2, 2), dtype=int32)


### Inner and outer products of vectors

In [9]:
# For some reason both matmult, tensordot, and einsum crash the GPU and kill the kernel
# if I try to do inner or outer product of vectors.
# All other matrix operations (with non-vector matrices) work!

a = tf.Variable([1.0, 2.0])
b = tf.Variable(a)
# print('b dot a=', tf.matmul(a, b, transpose_a=True))

# Returns: Blas GEMM launch failed : a.shape=(1, 2), b.shape=(1, 2), m=2, n=2, k=1 [Op:MatMul]
# Then hangs and kills the kernel.

In [10]:
# It works on a CPU:

with tf.device('CPU:0'):
    a = tf.Variable([[1.0, 2.0]])
    b = tf.Variable(a)
    print('outer product=', tf.matmul(a, b, transpose_a=True))
    print('inner product=', tf.matmul(a, b, transpose_b=True))
    
    a = tf.Variable([1.0, 2.0])
    b = tf.Variable(a)
    print('dotproduct=', tf.tensordot(a, b, axes=0))

outer product= tf.Tensor(
[[1. 2.]
 [2. 4.]], shape=(2, 2), dtype=float32)
inner product= tf.Tensor([[5.]], shape=(1, 1), dtype=float32)
dotproduct= tf.Tensor(
[[1. 2.]
 [2. 4.]], shape=(2, 2), dtype=float32)


In [11]:
# Versions that don't crash the GPU (with elementwise operations)
# With matrix vectors (1,2)

a = tf.Variable([[1.0, 2.0]])
b = tf.Variable(a)
print('a=', a)
print('b=', b)
print('inner:', tf.reduce_sum(a*b))
print('outer:', a*tf.transpose(b))

a= <tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[1., 2.]], dtype=float32)>
b= <tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[1., 2.]], dtype=float32)>
inner: tf.Tensor(5.0, shape=(), dtype=float32)
outer: tf.Tensor(
[[1. 2.]
 [2. 4.]], shape=(2, 2), dtype=float32)


In [12]:
# Same, but with vector vectors ( ,2)

a = tf.Variable([1.0, 2.0])
b = tf.Variable(a)
print('a=', a)
print('b=', b)
print('inner:', tf.reduce_sum(a*b))
print('outer:', a*tf.transpose(b), "- doesn't work in this case as transpose doesn't aply to flat vectors")
print('broadcasted outer:', a[None,:]*b[:,None])

a= <tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1., 2.], dtype=float32)>
b= <tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1., 2.], dtype=float32)>
inner: tf.Tensor(5.0, shape=(), dtype=float32)
outer: tf.Tensor([1. 4.], shape=(2,), dtype=float32) - doesn't work in this case as transpose doesn't aply to flat vectors
broadcasted outer: tf.Tensor(
[[1. 2.]
 [2. 4.]], shape=(2, 2), dtype=float32)


In [13]:
# Einstein notation.
# Still for some reason crashes for vectors-as-matrices :(
# But works for non-singleton-dimensioned matrices!

a = tf.Variable([[1,2], [3,4]])
b = tf.Variable([[0,1], [-1, 0]])
print(tf.einsum('ji,ki->jk', a, b))
print(tf.einsum('ij,ki->jk', a, b))
print(tf.einsum('ji,jk->jk', a, b))
print(tf.einsum('ij,jk->jk', a, b))
print(tf.einsum('ji,ik->j', a, b))

tf.Tensor(
[[ 2 -1]
 [ 4 -3]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 3 -1]
 [ 4 -2]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 0  3]
 [-7  0]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 0  4]
 [-6  0]], shape=(2, 2), dtype=int32)
tf.Tensor([-1 -1], shape=(2,), dtype=int32)


In [14]:
a = tf.Variable([1.0, 2.0])
# tf.einsum('ijk,ijk->ik', a[None,:,None], a[None,:,None])

# This still hangs the GPU. Even with broadcasting, it still does it :(((